# Sistemas de Recomendación

## Carga de Librerias

In [ ]:
import numpy as np
import pandas as pd
import scipy
import sys
import re
#nltk.download()
import nltk
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# pip install unidecode
sys.path.append(r"Utilities")
import Utilities 

## Lectura DataSet de Juguetes (usar encoding='iso-8859-1')

## Breve Análsis descriptivo

##### Conclusiones
- Hay 5 columnas de tipo string
- Se tiene un dataset de 5597x5
- No hay missing values

## Listado de opciones para recomendar (Listado juguetes)

## Cálculo vectores descriptivos de Items

##### Limpieza de datos

In [ ]:
#Tokenization


In [ ]:
# Eliminar palabras con ausencia de significaddo (remover stopwords)


In [ ]:
# Eliminar signos de puntuación (remover signos de puntuacion)


In [ ]:
# Union descripciones (pasar de array a string)


##### Cálculo Matriz Frecuencias (CountVectorizer(), usar encoding='iso-8859-1')

##### Visualización Matriz Frecuencias

##### Porcentaje de Ceros del total de elementos de la Matriz Frecuencias

##### Número de Ceros en cada fila

##### Número de Ceros en cada columna

##### Ejemplo de los valores diferentes de cero

##### Cálculo término frecuencia inversa de documento (TfidfTransformer())

https://mldeeplearning.com/text-vectorization-term-frequency-inverse-document-frequency-tfidf/

##### Visualizacion Matriz con términos frecuencia inversa de documento y normalizada

##### Ejemplo de los valores diferentes de cero

## Sistema Recomendación por Similitud entre items

Para cada objeto se calculan las similitud con el resto de objetos.

<br>
<br>

$$Similitud_{item} (x_1, x_2) = cos (\theta) = \frac {v_{x_1} \cdot v_{x_2}}{||v_{x_1}|| \cdot ||v_{x_2}||} = \frac {\sum_{i=1}^N (v_{x_1} * v_{x_2})}{\sum_{i=1}^N(v_{x_1,i}^2) * \sum_{i=1}^N(v_{x_2,i}^2)}$$

<br>

Para cada objeto se recomiendan los objetos con similitud más elevada.

<br>

##### Cálculo matriz similitudes

##### Validación de que la matriz anterior estaba normalizada (ver diagonal)

##### Visualizacion Matriz Similitudes

##### Ejemplo Recomendación por Similitud entre items

## Sistema Recomendación Filtrado Colaborativo Item-Item

Para cada usuario se propagan las valoraciones de cada objeto al resto mediante la métrica de similaridad entre los objetos

$$Score(u,x_i) = \sum\limits_{\substack{j=1 \\ j \neq i, \hspace{1mm} j \hspace{0.5mm} \epsilon \hspace{0.5mm} J}}^n w(rank_u(x_j)) * similitud_{item}(x_j, x_i)$$

Donde:
<ul>
    <li>$J$ es el conjunto de objetos valorados por el usuario $u$</li>
    <br>
    <li>$rank_u(x_j)$ es la valoración del usuario $u$ al objeto $x_j$</li>
    <br>
    <li>$w:[0-10] \to \mathbb{R}$ es una función que asigna a cada valoración un valor real</li>
</ul>

##### Simulación valoracion individual de un usuario

In [ ]:
# Simulamos valoraciones del 1 al 100

# 90 % valoraciones iguales a 0 (NA's)

# Cálculos previos valoración real


# Valoración normalizada


##### Ejemplo Recomendación de 10 juguetes por Filtrado Colaborativo Item-Item

In [ ]:
# Pasamos a DataFrame las puntuaciones predichas para cada juguete

# Solo seleccionamos los juguetes que no han sido valorados por el usuario (Se supone que los valorados ya han sido comprados por el usuario)

# Ordenación decreciente

# Muestra el top 10 de los juguetes recomendados 


## Sistema Recomendación Filtrado Colaborativo User-User

Permite estimar valoraciones de objetos que hayan valorado otros usuarios que tengan valoraciones en común con el usuario objetivo.

$$R(u_a, x) = \frac{\sum_{u_k \epsilon R_x} (z_{u_k, x} * similitud(u_a, u_k))}{\sum_{u_k \epsilon R_x} |similitud(u_a, u_k)|} * \sigma_a + \mu_a$$

$$similitud(u_a, u_k) = \frac{\sum_{i \epsilon L_{a,k}} (\upsilon_{k,i} - \mu_k)(\upsilon_{a,i} - \mu_a)}{\sqrt{\sum_{i \epsilon L_{a,k}} (\upsilon_{k,i} - \mu_k)^2} \sqrt{\sum_{i \epsilon L_{a,k}} (\upsilon_{a,i} - \mu_a)^2}}$$

$$z_{u_k, x} = \frac{\upsilon_{k, x} - \mu_k}{\sigma_k}$$

Donde:
<ul>
    <li>$u_a$ denota al usuario $a$</li>
    <li>$u_k$ denota al usuario $k$</li>
    <li>$x$ denota el objeto $x$ a valorar</li>
    <li>$R_x$ es el conjunto de usuarios que han valorado el objeto $x$</li>
    <li>$z_{u_k, x}$ es la normalización de la valoración del usuario $k$ sobre el objeto $x$</li>
    <li>$\sigma_a$ denota la desviación típica de las valoraciones del usuario $a$</li>
    <li>$\mu_a$ denota la media aritmética de las valoraciones del usuario $a$</li>
    <li>$\mu_k$ denota la media aritmética de las valoraciones del usuario $k$</li>
    <li>$L_{a,k}$ es el listado de objetos que han valorado tanto el usuario $a$ como el usuario $k$</li>
    <li>$\upsilon_{k,i}$ es la valoración media realizada por el usuario $k$ sobre el objeto $i$</li>
    <li>$\upsilon_{a,i}$ es la valoración media realizada por el usuario $a$ sobre el objeto $i$</li>    
</ul>

##### Lectura csv con valoraciones de 4 usuarios

In [ ]:
valoracionesUsuarios = pd.read_csv("DatosJuguetes\ValoracionesUsuarios.csv", delimiter=";", encoding='iso-8859-1')
valoracionesUsuarios = valoracionesUsuarios.iloc[:, [0,2,3,4,5]]
valoracionesUsuarios.head(10)

##### Ejemplo valoraciones user1

In [ ]:
valoracionesUsuarios.iloc[:,1][np.where(valoracionesUsuarios.iloc[:,1].notnull())[0]]

##### Cálculo valoraciones normalizadas 

In [ ]:
# Cálculos previos valoración real (valoracioes normalizadas)
medias = valoracionesUsuarios.iloc[:,1:5].apply(lambda x: x[np.where(x.notnull())[0]].mean(), axis=0)
desviaciones = valoracionesUsuarios.iloc[:,1:5].apply(lambda x: x[np.where(x.notnull())[0]].std(), axis=0)

# Normalización valoraciones
valoracionesUsuariosNorm = (valoracionesUsuarios.iloc[:,1:valoracionesUsuarios.shape[1]] - medias.values)/desviaciones
valoracionesUsuariosNorm = valoracionesUsuariosNorm.fillna(value=0)
valoracionesUsuariosNorm

##### Cálculo Similitud entre usuarios

##### Cálculo Scores Recomendación

In [ ]:
Numerador = valoracionesUsuariosNorm.dot(SimilUsuarios)
Denominador = (valoracionesUsuariosNorm != 0).dot(abs(SimilUsuarios))
Cociente = (Numerador/Denominador).values
Cociente

##### Ejemplo Recomendación juguetes por Filtrado Colaborativo User-User

In [ ]:
RecomendacionFiltradoColaborativoUserUser = pd.DataFrame(data = Cociente, index = Listado.values)

# Ejemplo:
# Se selecciona usuario
Usuario = 4
# Se seleccionan aquellos elemento que no estan valorados por el usuario 4 y que tienen valores predichos por Filtrado Colaboratibo User-User
ElemntosValidos = RecomendacionFiltradoColaborativoUserUser.iloc[:, (Usuario - 1)].notnull().values * valoracionesUsuarios.iloc[:, Usuario].isnull().values

# Recomendación
RecomendacionFiltradoColaborativoUserUser = RecomendacionFiltradoColaborativoUserUser.iloc[ElemntosValidos, (Usuario - 1)]
pd.DataFrame(RecomendacionFiltradoColaborativoUserUser.iloc[(-RecomendacionFiltradoColaborativoUserUser).argsort().values].index.values, columns = ['Juguete'])

## Sistema Recomendación Reducción Dimensionalidad 


Surprise es una librería específicamente diseñada para construir sistemas de recomendación en Python. Puede considerarse como la versión de scikit-learn para algoritmos de recomendación. `surprise` se ha convertido en una de las librerías para algoritmos de recomendacón más populares en python. Esto se debe a que es extremadamente robusto y fácil de usar.

La gran ventaja es que nos ofrece implementaciones de los algoritmos de filtro colaborativo más populares. Además también nos permitiria incorporar nuestro propio algoritomo dentro de su framework.

<center><img src='https://image.slidesharecdn.com/4772391/95/building-a-recommendation-engine-an-example-of-a-product-recommendation-engine-21-728.jpg' width=600 ></center>

In [ ]:
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import cross_validate, train_test_split
from surprise import SVD

# Un reader es todavia necesario pero solo parametro rating_scale es requerido
reader = Reader(rating_scale=(1, 5))

df = pd.read_csv("DatosJuguetes/u.data", delimiter="\t", header = None, encoding='iso-8859-1')
df.columns= ['userID', 'itemID', 'rating', 'timestamp']
# Las columnas deben corresponder a user id, item id and ratings (en este orden).
data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)

# Metodo seleccionado
modelo=SVD()

In [ ]:
df

In [ ]:
# Cross validation
cross_validate(modelo, data, cv=5)

In [ ]:
# train and tets
trainset, testset = train_test_split(data, test_size=.25)

# Entrenamiento y predicción
modelo.fit(trainset)
predictions = modelo.test(testset)
predictions

In [ ]:
# Accuracy RMSE
accuracy.rmse(predictions)

## Más algoritmos en el paquete `surprise`

No vamos a entrar en detalles del resto de algoritmos que contiene el paquete, pero vamos a listarlos para que los conozcaís y valoreis su utilización en el futuro.




### NormalPredictor

NormalPredictor predice los ratings de forma aleatoria entendiendo que la distribución de las puntuaciones sigue una normal. Es uno de los algoritmos más básicos que podemos encontrar.




### BaselineOnly
BaselineOnly nos devolverá el baseline para la combinación de usuario e ítem.




### k-NN algorithms

#### KNNBasic
KNNBasic es un algoritmo básico de filtro colaborativo.



#### KNNWithMeans
KNNWithMeans es un algoritmo de filtro colaborativo, que tienen en cuenta la media de los ratings de cada usuario. 



#### KNNWithZScore
KNNWithZScore es un algoritmo de filtro colaborativo, que tienen en cuenta la normalización z-score de las puntuaciones de cada usuario.



#### KNNBaseline
KNNBaseline es un algoritmo de filtro colaborativo, basado en el baseline. 


### Matrix Factorization-based algorithms


#### SVD y SVDpp

SVD y SVDpp son dos versiones del algoritmo que hemos analizado anteriormente.



#### NMF

NMF  es un algoritmo de filtro colaborativo basado en Non-negative Matrix Factorization. Conceptualmente es muy similar a  SVD.


### Slope One
SlopeOne es una implementación del algoritmo SlopeOne. https://en.wikipedia.org/wiki/Slope_One



### Co-clustering

Algortimo de filtro colaborativo basado en co-clustering.